# Inception models

In [ ]:
import os,sys
import cv2
from tqdm import tqdm
import re
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical, image_dataset_from_directory

# VGG16 import
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16

# VGG19 import
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input as preprocess_input_vgg19

# inception_v3 import
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_incv3, InceptionV3

#extra
from tensorflow.keras.layers import Input

In [ ]:
# path for kaggle
train_dir = '/kaggle/input/stanford-dogs-dataset-traintest/cropped/train'
test_dir = "/kaggle/input/stanford-dogs-dataset-traintest/cropped/test"

In [ ]:
# load train
from tensorflow.keras.utils import image_dataset_from_directory

train_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

#preprocess train
prep_train_ds = train_ds.map(lambda x, y: (preprocess_input_incv3(x), y))

In [ ]:
# load val
validation_ds = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32)

#preprocess val
prep_val_ds = validation_ds.map(lambda x, y: (preprocess_input_incv3(x), y))

In [ ]:
# load test
test_ds = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(224, 224),
    batch_size=32)

#preprocess test
prep_test_ds = test_ds.map(lambda x, y: (preprocess_input_incv3(x), y))

In [ ]:
input_shape = train_ds.element_spec[0].shape[1:]
input_tensor = Input(shape=input_shape)

base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)
# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

## 2 dense 100, 86% test acc

In [ ]:
def model_init():
    augmentation = Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.2, 0.2),
        layers.RandomRotation(0.1)
    ])

    augment_model = Sequential([
        layers.Input(shape = input_shape),
        augmentation,
        base_model,
        layers.Flatten(),
        layers.Dense(100, activation="relu"),
        layers.Dense(100, activation="relu"),
        layers.Dense(120, activation='softmax')
    ])
    return augment_model

In [ ]:
augment_model = model_init()
augment_model.summary()

In [ ]:
opt = optimizers.Adam(learning_rate = 1e-4)

def compile_model(model):
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
augment_model = compile_model(augment_model)

MODEL = "model_1"

epochs=20

es = EarlyStopping(monitor='val_loss', 
                   mode='auto', 
                   patience=5, 
                   verbose=1, 
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor = 0.1,
                       patience=3,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint("{}.h5".format(MODEL),
                      save_weights_only=True,
                      monitor='val_accuracy',
                      mode='max',
                      verbose=0,
                      save_best_only=True)

history = augment_model.fit(prep_train_ds, 
                        validation_data=prep_val_ds, 
                        epochs=30,
                        callbacks=[es, lr, mcp],
                        batch_size=32,
                        verbose=1)

In [ ]:
loss, accuracy = augment_model.evaluate(prep_test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(30)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 1 dense 256

In [ ]:
def model_init():
    augmentation = Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.2, 0.2),
        layers.RandomRotation(0.1)
    ])

    augment_model_1 = Sequential([
        layers.Input(shape = input_shape),
        augmentation,
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(120, activation='softmax')
    ])
    return augment_model_1

In [ ]:
augment_model_1 = model_init()
augment_model_1.summary()

In [ ]:
opt = optimizers.Adam(learning_rate = 1e-4)

def compile_model(model):
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
augment_model_1 = compile_model(augment_model_1)

MODEL = "model_1"

epochs=20

es = EarlyStopping(monitor='val_loss', 
                   mode='auto', 
                   patience=5, 
                   verbose=1, 
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor = 0.1,
                       patience=3,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint("{}.h5".format(MODEL),
                      save_weights_only=True,
                      monitor='val_accuracy',
                      mode='max',
                      verbose=0,
                      save_best_only=True)

history_1 = augment_model_1.fit(prep_train_ds, 
                        validation_data=prep_val_ds, 
                        epochs=30,
                        callbacks=[es, lr, mcp],
                        batch_size=32,
                        verbose=1)

In [ ]:
loss, accuracy = augment_model_1.evaluate(prep_test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
acc = history_1.history['accuracy']
val_acc = history_1.history['val_accuracy']

loss = history_1.history['loss']
val_loss = history_1.history['val_loss']

epochs_range = range(19)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## third go

In [ ]:
def model_init():
    augmentation = Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.2, 0.2),
        layers.RandomRotation(0.1)
    ])

    augment_model_2 = Sequential([
        layers.Input(shape = input_shape),
        augmentation,
        base_model,
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(120, activation='softmax')
    ])
    return augment_model_2

In [ ]:
augment_model_2 = model_init()
augment_model_2.summary()

In [ ]:
opt = optimizers.Adam(learning_rate = 1e-4)

def compile_model(model):
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
augment_model_2 = compile_model(augment_model_1)

MODEL = "model_3_128"

epochs=20

es = EarlyStopping(monitor='val_loss', 
                   mode='auto', 
                   patience=4, 
                   verbose=1, 
                   restore_best_weights=True)

lr = ReduceLROnPlateau(monitor="val_loss",
                       factor = 0.1,
                       patience=2,
                       verbose=1,
                       min_lr=0)

mcp = ModelCheckpoint("{}.h5".format(MODEL),
                      save_weights_only=True,
                      monitor='val_accuracy',
                      mode='max',
                      verbose=0,
                      save_best_only=True)

history_2 = augment_model_2.fit(prep_train_ds, 
                        validation_data=prep_val_ds, 
                        epochs=30,
                        callbacks=[es, lr, mcp],
                        batch_size=32,
                        verbose=1)

In [ ]:
loss, accuracy = augment_model_2.evaluate(prep_test_ds)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
acc = history_2.history['accuracy']
val_acc = history_2.history['val_accuracy']

loss = history_2.history['loss']
val_loss = history_2.history['val_loss']

epochs_range = range(23)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()